In [1]:
import pandas as pd

### 상장 목록 받아오기

In [2]:
df = pd.read_html('https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]

In [3]:
df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,4840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시


In [4]:
df['종목코드'] = df['종목코드'].map('{:06d}'.format)

In [5]:
df = df.sort_values(by='종목코드')

In [6]:
df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
955,동화약품,000020,의약품 제조업,"의약품 제조,판매,수출입",1976-03-24,12월,유준하,http://www.dong-wha.co.kr,서울특별시
321,KR모터스,000040,그외 기타 운송장비 제조업,"이륜차(오토바이) 제조,도매",1976-05-25,12월,노성석,http://www.krmotors.com,경상남도
1584,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준, 김담",http://www.kyungbang.co.kr,서울특별시
1606,메리츠화재,000060,보험업,"손해보험(화재,해상,운송보험)",1956-07-02,12월,김용범,http://www.meritzfire.com,서울특별시
1610,삼양홀딩스,000070,기타 금융업,"지주회사,경영자문컨설팅",1968-12-27,12월,"엄태웅, 윤재엽",http://www.samyang.com,서울특별시


### 일별 시세 구하기

In [7]:
from bs4 import BeautifulSoup
import requests

In [8]:
code = '005930' # 삼성전자 코드
url = f'https://finance.naver.com/item/sise_day.naver?code={code}'
req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
html = BeautifulSoup(req.text, "lxml")

In [9]:
headers = {'User-agent': 'Mozilla/5.0'}

In [10]:
pgrr = html.find('td', class_='pgRR')
print(pgrr.a['href'])

/item/sise_day.naver?code=005930&page=647


In [11]:
# .prettify() 통해 getText 속성값을 계층적으로 보기좋게 출력 가능
print(pgrr.prettify())

<td class="pgRR">
 <a href="/item/sise_day.naver?code=005930&amp;page=647">
  맨뒤
  <img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
 </a>
</td>



In [12]:
print(pgrr.text)


맨뒤
				




In [13]:
s = pgrr.a['href'].split('=')
print(s)

['/item/sise_day.naver?code', '005930&page', '647']


In [14]:
last_page = s[-1]
print(last_page)

647


In [15]:
code = '005930' # 삼성전자 코드
url = f'https://finance.naver.com/item/sise_day.naver?code={code}'
df = None

for page in range(1, int(last_page)+1):
    req = requests.get(f'{url}&page={page}', headers=headers)
    df = pd.concat([df, pd.read_html(req.text, encoding = 'euc-kr')[0]], ignore_index=True)

In [16]:
# 데이터가 없는 행 일괄 삭제
df.dropna(inplace = True)

In [17]:
# 인덱스 재배열
df.reset_index(drop=True, inplace=True)

In [18]:
df # 삼성전자 일별 시세

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.03.23,70500.0,200.0,70600.0,71200.0,70300.0,12332242.0
1,2022.03.22,70300.0,400.0,69900.0,70500.0,69900.0,9402666.0
2,2022.03.21,69900.0,800.0,70900.0,71000.0,69900.0,11169002.0
3,2022.03.18,70700.0,500.0,70600.0,70900.0,70200.0,14410038.0
4,2022.03.17,71200.0,800.0,71200.0,71800.0,70900.0,17646315.0
...,...,...,...,...,...,...,...
6458,1996.06.29,68500.0,400.0,68100.0,69100.0,67100.0,96710.0
6459,1996.06.28,68100.0,1200.0,67300.0,68500.0,67200.0,138430.0
6460,1996.06.27,66900.0,800.0,67500.0,67700.0,66700.0,155450.0
6461,1996.06.26,67700.0,200.0,67600.0,67900.0,66000.0,136630.0


In [19]:
df.to_excel("samsung_price.xlsx", index=False)